In [ ]:
import docker
client = docker.from_env()

In [ ]:
container = client.containers.run("alpine", "sleep 100", remove=True, detach=True)
result = container.exec_run("cat", stdin=True, socket=True)
socket = result.output

In [23]:
socket._sock.sendall(b"Hello World\n")

In [25]:
socket._sock.recv(1024)

b''

In [29]:
socket.close()

In [33]:
container.id

'47e6d89bd0ddfcbaa524bbd83142cc2a19e99af871e91f97eeddbfd211c0f41f'

In [ ]:
import socket
code = """\
#include <iostream>
int main()
{
  std::cout << "Hello World";
  return 0;
}
"""

container = client.containers.run("gcc:15.1.0-bookworm", "tail -f /dev/null", remove=True, detach=True)
res = container.exec_run(
    "sh -c \"g++ --std=c++20 -o main -x c++ - && ./main\"",
    workdir="/usr/src",
    stdin=True, stdout=True, stderr=True, socket=True
)

sock = res.output._sock
sock.sendall(code.encode())
sock.shutdown(socket.SHUT_WR)
while chunk:=sock.recv(1024):
    print(chunk.decode(), end="")

      Lcc1plus: fatal error: &&: No such file or directory
compilation terminated.
      Pcc1plus: fatal error: ./main: No such file or directory
compilation terminated.


In [ ]:
sock.write()

ValueError: I/O operation on closed file.